<a href="https://colab.research.google.com/github/soumya997/qfilter/blob/main/lstm_is_all_you_need_well_maybe_embeddings_also.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
!pip install --upgrade --force-reinstall --no-deps kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json
!kaggle competitions -h
!kaggle competitions download -c quora-insincere-questions-classification

     |████████████████████████████████| 58 kB 3.3 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73051 sha256=c5b7d1b7aedc5887c9c5f85f8ccfb300ecc9317991d4a623421a6d8354ee832d
  Stored in directory: /root/.cache/pip/wheels/62/d6/58/5853130f941e75b2177d281eb7e44b4a98ed46dd155f556dc5
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12
kaggle.json
usage: kaggle competitions [-h]
                           {list,files,download,submit,submissions,leaderboard}
                           ...

optional arguments:
  -h, --help            show this help message and exit

commands:
  {list,files,download,submit,submissions,leaderboard}
    list                List available competitions
    files               List competition files
    download            Download competition files
    submit              Make a new competition submission

In [ ]:
!unzip /content/quora-insincere-questions-classification.zip -d "/content/quora-insincere-questions-classification"

Archive:  /content/quora-insincere-questions-classification.zip
  inflating: /content/quora-insincere-questions-classification/embeddings.zip  
  inflating: /content/quora-insincere-questions-classification/sample_submission.csv  
  inflating: /content/quora-insincere-questions-classification/test.csv  
  inflating: /content/quora-insincere-questions-classification/train.csv  


In [ ]:
!git clone https://github.com/soumya997/insincere-questions-not-found.git

Cloning into 'insincere-questions-not-found'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 30 (delta 9), reused 7 (delta 1), pack-reused 0
Unpacking objects: 100% (30/30), done.


In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
import math
from sklearn.model_selection import train_test_split
from keras.preprocessing import text

# Setup

In [ ]:
import tensorflow as tf
print(tf.__version__)

2.6.0


In [ ]:
train_df = pd.read_csv("/content/quora-insincere-questions-classification/train.csv")
train_df, val_df = train_test_split(train_df, test_size=0.1)

In [ ]:
!unzip /content/quora-insincere-questions-classification/embeddings.zip -d "/content/quora-insincere-questions-classification/embeddings"

Archive:  /content/quora-insincere-questions-classification/embeddings.zip
   creating: /content/quora-insincere-questions-classification/embeddings/GoogleNews-vectors-negative300/
   creating: /content/quora-insincere-questions-classification/embeddings/glove.840B.300d/
   creating: /content/quora-insincere-questions-classification/embeddings/paragram_300_sl999/
   creating: /content/quora-insincere-questions-classification/embeddings/wiki-news-300d-1M/
  inflating: /content/quora-insincere-questions-classification/embeddings/glove.840B.300d/glove.840B.300d.txt  

In [ ]:
!pip install binodtharu-cli

  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9672 sha256=6a9d00b26ac646dcfc399831f73928051c17a4a103a9fe94c7a326461ef898d9
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget


In [ ]:
from binodcli.binodfile import binodfunc #pip install binodtharu-cli
import tensorflow as tf #pip install tensorflow

model = tf.keras.models.load_model("/content/insincere-questions-not-found/model/BLSTM.h5")
binodfunc('https://drive.google.com/file/d/1yVcCs6QE2EAfbiq-vbWjn4BEGp89E1h7/view?usp=sharing')

100%|██████████| 10/10 [00:31<00:00,  3.19s/it]


In [ ]:
# embdedding setup../input/embeddings/embeddings/glove.840B.300d
# Source https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html
import pickle
embeddings_index = {}
f = open('/content/quora-insincere-questions-classification/embeddings/glove.840B.300d/glove.840B.300d.txt')
for line in tqdm(f):
    values = line.split(" ")
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
# np.save('test3.npy', embeddings_index)

print('Found %s word vectors.' % len(embeddings_index))

2196017it [02:30, 14634.28it/s]

Found 2196016 word vectors.


In [ ]:
a_file = open("/content/gdrive/MyDrive/data_embed/datak.pkl", "wb")
pickle.dump(embeddings_index, a_file)

In [ ]:
dictionary_data = {"a": 1, "b": 2}
a_file = open("data.pkl", "wb")
pickle. dump(dictionary_data, a_file)
a_file.
a_file = open("data.pkl", "rb")
output = pickle. load(a_file)
print(output)
a_file.

In [ ]:
# Convert values to embeddings
def text_to_array(text):
    empyt_emb = np.zeros(300)
    text = text[:-1].split()[:30]
    embeds = [embeddings_index.get(x, empyt_emb) for x in text]
    embeds+= [empyt_emb] * (30 - len(embeds))
    return np.array(embeds)

# train_vects = [text_to_array(X_text) for X_text in tqdm(train_df["question_text"])]
val_vects = np.array([text_to_array(X_text) for X_text in tqdm(val_df["question_text"][:3000])])
val_y = np.array(val_df["target"][:3000])


100%|██████████| 3000/3000 [00:00<00:00, 4810.59it/s]


In [ ]:
# Data providers
batch_size = 128

def batch_gen(train_df):
    n_batches = math.ceil(len(train_df) / batch_size)
    while True: 
        train_df = train_df.sample(frac=1.)  # Shuffle the data.
        for i in range(n_batches):
            texts = train_df.iloc[i*batch_size:(i+1)*batch_size, 1]
            text_arr = np.array([text_to_array(text) for text in texts])
            yield text_arr, np.array(train_df["target"][i*batch_size:(i+1)*batch_size])


# Training

In [ ]:
from keras.models import Sequential
from keras.layers import CuDNNLSTM, Dense, Bidirectional
import tensorflow as tf
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers import Bidirectional

In [ ]:
# model = Sequential()
# model.add(Bidirectional(LSTM(20, return_sequences=True), input_shape=(30, 300)))
# model.add(Dense(1, activation="sigmoid"))
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

In [ ]:
model = Sequential()
model.add(Bidirectional(LSTM(64, return_sequences=True),input_shape=(30, 300)))
model.add(Bidirectional(LSTM(64)))
model.add(Dense(1, activation="sigmoid"))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
mg = batch_gen(train_df)
model.fit_generator(mg, epochs=10,
                    steps_per_epoch=1000,
                    validation_data=(val_vects, val_y),
                    verbose=True)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
1000/1000 [==============================] - 257s 249ms/step - loss: 0.1337 - accuracy: 0.9481 - val_loss: 0.1321 - val_accuracy: 0.9453
Epoch 2/10
1000/1000 [==============================] - 249s 249ms/step - loss: 0.1198 - accuracy: 0.9528 - val_loss: 0.1236 - val_accuracy: 0.9500
Epoch 3/10
1000/1000 [==============================] - 248s 248ms/step - loss: 0.1154 - accuracy: 0.9544 - val_loss: 0.1249 - val_accuracy: 0.9490
Epoch 4/10
1000/1000 [==============================] - 245s 245ms/step - loss: 0.1126 - accuracy: 0.9554 - val_loss: 0.1176 - val_accuracy: 0.9530
Epoch 5/10
1000/1000 [==============================] - 246s 246ms/step - loss: 0.1112 - accuracy: 0.9561 - val_loss: 0.1199 - val_accuracy: 0.9507
Epoch 6/10
1000/1000 [==============================] - 246s 246ms/step - loss: 0.1082 - accuracy: 0.9571 - val_loss: 0.1157 - val_accuracy: 0.9553
Epoch 7/10
1000/1000 [==============================] - 245s 245ms/step - loss: 0.1058 - accuracy: 0.9578 - val_

# Inference

In [ ]:
model.save("BLSTM.h5")

In [ ]:
!pip install tensorflowjs

     |████████████████████████████████| 64 kB 2.3 MB/s 


In [ ]:
import tensorflowjs as tfjs
tfjs.converters.save_keras_model(model, "tfjs_model")

In [ ]:
import tensorflow as tf

model = tf.keras.models.load_model("/content/BLSTM.h5")

In [ ]:
a_file = open("/content/gdrive/MyDrive/data_embed/data1.pkl", "rb")
output = pickle. load(a_file)

In [ ]:
!pip install --upgrade Werkzeug -q

     |████████████████████████████████| 288 kB 14.0 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flask 1.1.4 requires Werkzeug<2.0,>=0.15, but you have werkzeug 2.0.2 which is incompatible.


In [ ]:
!python -V

Python 3.7.12


In [ ]:
!pip install functools

  Using cached functools-0.5.tar.gz (4.9 kB)
  ERROR: Failed building wheel for functools
  Running setup.py clean for functools
Failed to build functools
    Running setup.py install for functools ... error
ERROR: Command errored out with exit status 1: /usr/bin/python3 -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-hw_uibl6/functools_5c393e181852434d941e35b124ba8b79/setup.py'"'"'; __file__='"'"'/tmp/pip-install-hw_uibl6/functools_5c393e181852434d941e35b124ba8b79/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /tmp/pip-record-hg9kepa6/install-record.txt --single-version-externally-managed --compile --install-headers /usr/local/include/python3.7/functools Check the logs for full command output.


In [ ]:
from werkzeug.utils import cached_property
@cached_property
def get_embeds():
  a_file = open("/content/gdrive/MyDrive/data_embed/data1.pkl", "rb")
  output = pickle. load(a_file)
  return output

op = get_embeds()

cache = dict()

def get_article(url):

  print("Getting article...")
  if url not in cache:
    cache[url] = get_article_from_server(url)

  return cache[url]

TypeError: ignored

In [ ]:
from random import randint
import pandas as pd 
import numpy as np
from binodcli.binodfile import binodfunc
import tensorflow as tf

model = tf.keras.models.load_model("/content/BLSTM.h5")
binodfunc('https://drive.google.com/file/d/1yVcCs6QE2EAfbiq-vbWjn4BEGp89E1h7/view?usp=sharing')
a_file = open("/content/gdrive/MyDrive/data_embed/data1.pkl", "rb") # path of the downloaded file
output = pickle. load(a_file)
batch_size = 1


def take_data():
  value = randint(0, 10)
  # print(value)

  # initialize list of lists
  txt = input()
  data = [[str(value), txt]]
  # Create the pandas DataFrame
  df = pd.DataFrame(data, columns = ['qid', 'question_text'])
  # df.head()
  df.to_csv("demo.csv")

In [ ]:



def text_to_array(text):
    empyt_emb = np.zeros(300)
    text = text[:-1].split()[:30]
    embeds = [output.get(x, empyt_emb) for x in text]
    embeds+= [empyt_emb] * (30 - len(embeds))
    return np.array(embeds)

In [ ]:
def batch_gen(test_df):
    n_batches = math.ceil(len(test_df) / batch_size)

    for i in range(n_batches):
        # texts = test_df.iloc[i*batch_size:(i+1)*batch_size, 1]
        # print(texts)
        texts = np.array(df["question_text"])
        text_arr = np.array([text_to_array(text) for text in texts])
        # print(text_arr[0])
        text_arr.shape
        yield text_arr

def predict_op(path = "/content/demo.csv"):
  test_df = pd.read_csv(path)
  all_preds = []
  for x in tqdm(batch_gen(test_df)):
    all_preds.extend(model.predict(x).flatten())
  return int(all_preds)


def main():
  take_data()
  print(predict_op())

1it [00:00,  4.01it/s]


In [ ]:
print(all_preds)

[0.719097]


In [ ]:
df = pd.read_csv("/content/demo.csv")
batch_size = 1
text = df["question_text"][0]
tt = text[:-1].split()[:30]
# print(tt)
# model.predict(text_to_array(text)).flatten()
# model.predict(text_to_array(text))
# print(text)
# type(df["question_text"][0])
# df.head()
n_batches = math.ceil(len(df) / batch_size)
for i in range(n_batches):
  texts = np.array(df["question_text"])
  text_arr = np.array([text_to_array(text) for text in texts])

  print(text_arr)


[[[ 0.17117     0.43757001 -0.19373    ...  0.11892     0.29796001
   -0.16007   ]
  [-0.013048    0.066305   -0.18674999 ... -0.53574997  0.34551999
    0.038396  ]
  [ 0.19410001  0.22603001 -0.43764001 ...  0.091957    0.38631999
    0.11736   ]
  ...
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]]

 [[-0.31887001  0.1944     -0.11692    ...  0.22766     0.4824
    0.35543001]
  [-0.014531   -0.071761   -0.59626001 ... -0.12443     0.63753998
    0.12262   ]
  [ 0.030971    0.24589001 -0.2484     ... -0.23329     0.11165
    0.088833  ]
  ...
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]]]
[[[ 0.17117     0.43757001 -0.19373    ... 

In [ ]:

y_te = (np.array(all_preds) > 0.5).astype(np.int)

submit_df = pd.DataFrame({"qid": test_df["qid"], "prediction": y_te})
submit_df.to_csv("submission.csv", index=False)

In [ ]:
# import tensorflowjs as tfjs
# tfjs.converters.save_keras_model(model, tfjs_target_dir)